In [1]:
ls

CoronaNet/                          SelfAnalysis.ipynb
Get_Chinese_Subnational_Data.ipynb


In [2]:
ls ../data

OxCGRT_US_states_temp.csv     OxCGRT_latest_responses.csv
OxCGRT_latest.csv             OxCGRT_latest_withnotes.csv
OxCGRT_latest_allchanges.csv  timeseries/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.max_columns', None)

#### First, we extra original data from `OxCGRT_latest_withnotes`

In [6]:
df_latest_withnotes = pd.read_csv('../data/OxCGRT_latest_withnotes.csv')
df_latest_withnotes.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CountryName,CountryCode,RegionName,RegionCode,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,C2_Flag,C2_Notes,C3_Cancel public events,C3_Flag,C3_Notes,C4_Restrictions on gatherings,C4_Flag,C4_Notes,C5_Close public transport,C5_Flag,C5_Notes,C6_Stay at home requirements,C6_Flag,C6_Notes,C7_Restrictions on internal movement,C7_Flag,C7_Notes,C8_International travel controls,C8_Notes,E1_Income support,E1_Flag,E1_Notes,E2_Debt/contract relief,E2_Notes,E3_Fiscal measures,E3_Notes,E4_International support,E4_Notes,H1_Public information campaigns,H1_Flag,H1_Notes,H2_Testing policy,H2_Notes,H3_Contact tracing,H3_Notes,H4_Emergency investment in healthcare,H4_Notes,H5_Investment in vaccines,H5_Notes,M1_Wildcard,M1_Notes,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,NaN,NaN,20200101,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,NaN,NaN,20200102,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,ABW,NaN,NaN,20200103,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,ABW,NaN,NaN,20200104,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,ABW,NaN,NaN,20200105,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_china_ox = df_latest_withnotes[df_latest_withnotes.CountryCode == 'CHN']
df_china_ox.head(2)

,CountryName,CountryCode,RegionName,RegionCode,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,C2_Flag,C2_Notes,C3_Cancel public events,C3_Flag,C3_Notes,C4_Restrictions on gatherings,C4_Flag,C4_Notes,C5_Close public transport,C5_Flag,C5_Notes,C6_Stay at home requirements,C6_Flag,C6_Notes,C7_Restrictions on internal movement,C7_Flag,C7_Notes,C8_International travel controls,C8_Notes,E1_Income support,E1_Flag,E1_Notes,E2_Debt/contract relief,E2_Notes,E3_Fiscal measures,E3_Notes,E4_International support,E4_Notes,H1_Public information campaigns,H1_Flag,H1_Notes,H2_Testing policy,H2_Notes,H3_Contact tracing,H3_Notes,H4_Emergency investment in healthcare,H4_Notes,H5_Investment in vaccines,H5_Notes,M1_Wildcard,M1_Notes,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
8960,China,CHN,NaN,NaN,20200101,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,Report of unknown virus in Wuhan https://web.a...,1.0,"""However, testing kits for the disease were no...",1.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,27.0,0.0,0.0,0.0,0.0,0.0,6.41,6.41,7.58,7.58,0.0,0.0
8961,China,CHN,NaN,NaN,20200102,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,1.0,NaN,1.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,27.0,0.0,0.0,0.0,0.0,0.0,6.41,6.41,7.58,7.58,0.0,0.0


In [9]:
df_china_ox.tail(2)

,CountryName,CountryCode,RegionName,RegionCode,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,C2_Flag,C2_Notes,C3_Cancel public events,C3_Flag,C3_Notes,C4_Restrictions on gatherings,C4_Flag,C4_Notes,C5_Close public transport,C5_Flag,C5_Notes,C6_Stay at home requirements,C6_Flag,C6_Notes,C7_Restrictions on internal movement,C7_Flag,C7_Notes,C8_International travel controls,C8_Notes,E1_Income support,E1_Flag,E1_Notes,E2_Debt/contract relief,E2_Notes,E3_Fiscal measures,E3_Notes,E4_International support,E4_Notes,H1_Public information campaigns,H1_Flag,H1_Notes,H2_Testing policy,H2_Notes,H3_Contact tracing,H3_Notes,H4_Emergency investment in healthcare,H4_Notes,H5_Investment in vaccines,H5_Notes,M1_Wildcard,M1_Notes,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
9238,China,CHN,NaN,NaN,20201005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90629.0,4739.0,NaN,45.83,NaN,48.81,NaN,52.88,NaN,55.68,NaN,37.5
9239,China,CHN,NaN,NaN,20201006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.83,NaN,48.81,NaN,52.88,NaN,55.68,NaN,37.5


#### The Chinese data from OxCGRT is only national

In [10]:
len(df_china_ox)

280

#### Our goal is to save a new csv file with Chinese Sub National Data, with each region's data being ranked by date. We can use the US data format as an example

In [11]:
df_us_ox = df_latest_withnotes[df_latest_withnotes.CountryCode == 'USA']
df_us_ox.head(2)

,CountryName,CountryCode,RegionName,RegionCode,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,C2_Flag,C2_Notes,C3_Cancel public events,C3_Flag,C3_Notes,C4_Restrictions on gatherings,C4_Flag,C4_Notes,C5_Close public transport,C5_Flag,C5_Notes,C6_Stay at home requirements,C6_Flag,C6_Notes,C7_Restrictions on internal movement,C7_Flag,C7_Notes,C8_International travel controls,C8_Notes,E1_Income support,E1_Flag,E1_Notes,E2_Debt/contract relief,E2_Notes,E3_Fiscal measures,E3_Notes,E4_International support,E4_Notes,H1_Public information campaigns,H1_Flag,H1_Notes,H2_Testing policy,H2_Notes,H3_Contact tracing,H3_Notes,H4_Emergency investment in healthcare,H4_Notes,H5_Investment in vaccines,H5_Notes,M1_Wildcard,M1_Notes,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
50120,United States,USA,NaN,NaN,20200101,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,Coded as 0 because unemployment insurance prog...,0.0,No national policies. Debt/contract relief may...,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50121,United States,USA,NaN,NaN,20200102,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df_us_ox.tail(2)

,CountryName,CountryCode,RegionName,RegionCode,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,C2_Flag,C2_Notes,C3_Cancel public events,C3_Flag,C3_Notes,C4_Restrictions on gatherings,C4_Flag,C4_Notes,C5_Close public transport,C5_Flag,C5_Notes,C6_Stay at home requirements,C6_Flag,C6_Notes,C7_Restrictions on internal movement,C7_Flag,C7_Notes,C8_International travel controls,C8_Notes,E1_Income support,E1_Flag,E1_Notes,E2_Debt/contract relief,E2_Notes,E3_Fiscal measures,E3_Notes,E4_International support,E4_Notes,H1_Public information campaigns,H1_Flag,H1_Notes,H2_Testing policy,H2_Notes,H3_Contact tracing,H3_Notes,H4_Emergency investment in healthcare,H4_Notes,H5_Investment in vaccines,H5_Notes,M1_Wildcard,M1_Notes,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
64958,United States,USA,Wyoming,US_WY,20201005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64959,United States,USA,Wyoming,US_WY,20201006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
len(df_us_ox) / len(set(df_us_ox.Date)) == len(set(df_us_ox.RegionName))

True

#### Great, so now we need to examine the data from CoronaNet and find the corresponding columns / policy code. 

In [17]:
ls CoronaNet/

README.md                      s41562-020-0909-7.pdf
coronanet_release_allvars.csv


In [19]:
df_corona_net = pd.read_csv("./CoronaNet/coronanet_release_allvars.csv")
df_corona_net.head()

,record_id,policy_id,entry_type,correct_type,update_type,update_level,description,date_announced,date_start,date_end,country,ISO_A3,ISO_A2,init_country_level,domestic_policy,province,city,type,type_sub_cat,type_text,school_status,target_country,target_geog_level,target_region,target_province,target_city,target_other,target_who_what,target_direction,travel_mechanism,compliance,enforcer,index_high_est,index_med_est,index_low_est,index_country_rank,link,date_updated,recorded_date
0,R_1gShvNnZtOkb4X1NA,3553952,new_entry,original,NaN,NaN,Bahrain Health Ministry monitoring coronavirus...,2020-01-22,2020-01-22,NaN,Bahrain,BHR,BH,National,0,NaN,NaN,Health Monitoring,NaN,NaN,NaN,All countries,All countries,NaN,NaN,NaN,NaN,1,Inbound/Outbound,All kinds of transport,Voluntary/Recommended but No Penalties,"National Government,Ministry/Department of Health",49.318559,47.307619,45.133747,124.0,https://www.moh.gov.bh/COVID19/Details/3790,2020-04-13,2020-04-13T12:30:56Z
1,R_3GrLmE8y71iEe17NA,2408864,new_entry,original,NaN,NaN,MoH of Barbados met in press conference to dis...,2020-01-22,2020-01-22,NaN,Barbados,BRB,BB,National,0,NaN,NaN,Public Awareness Measures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),Ministry/Department of Health,48.812998,46.853783,43.914687,79.0,https://gisbarbados.gov.bb/blog/health-ministr...,2020-04-20,2020-04-20T11:53:09Z
2,R_1Hi5XBHB1zBOJc7NA,5424670,new_entry,correction,NaN,NaN,"On January 22, 2020, Canada implements screeni...",2020-01-22,2020-01-22,NaN,Canada,CAN,CA,National,1,NaN,NaN,Health Monitoring,NaN,NaN,NaN,Canada,"One or more countries, but not all countries",NaN,NaN,NaN,NaN,All Residents (Citizen Residents +Foreign Resi...,Inbound,Flights,Mandatory (Unspecified/Implied),National Government,49.181419,47.440853,45.250540,135.0,https://www.canada.ca/en/public-health/service...,2020-04-01,2020-04-01T20:17:57Z
3,R_3EQBDJGMTKwowKOAq,2272584,new_entry,correction,NaN,NaN,China announced on January 22: to guide the pr...,2020-01-22,2020-01-22,NaN,China,CHN,CN,National,0,NaN,NaN,Public Awareness Measures,Disseminating information related to COVID-19 ...,NaN,NaN,China,NaN,NaN,NaN,NaN,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),"National Government,Ministry/Department of Hea...",52.655965,51.598420,50.520158,194.0,http://wjw.wuhan.gov.cn/ztzl_28/fk/tzgg/202004...,2020-07-20,2020-07-20T21:13:39Z
4,R_1OoGDBSpbZQ02oANA,5468900,new_entry,original,NaN,NaN,"Xi'an, China announced that subway stations an...",2020-01-22,2020-01-22,NaN,China,CHN,CN,Municipal,0,Shaanxi,Xi'an,Hygiene,NaN,NaN,NaN,China,NaN,NaN,Shaanxi,Xi'an,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),Municipal Government,52.655965,51.598420,50.520158,194.0,http://www.sxdaily.com.cn/2020-01/30/content_8...,2020-07-16,2020-07-16T07:39:56Z


In [21]:
# for now, we only want the chinese data
df_corona_net_china = df_corona_net[df_corona_net.country == 'China']
df_corona_net_china.head()

,record_id,policy_id,entry_type,correct_type,update_type,update_level,description,date_announced,date_start,date_end,country,ISO_A3,ISO_A2,init_country_level,domestic_policy,province,city,type,type_sub_cat,type_text,school_status,target_country,target_geog_level,target_region,target_province,target_city,target_other,target_who_what,target_direction,travel_mechanism,compliance,enforcer,index_high_est,index_med_est,index_low_est,index_country_rank,link,date_updated,recorded_date
3,R_3EQBDJGMTKwowKOAq,2272584,new_entry,correction,NaN,NaN,China announced on January 22: to guide the pr...,2020-01-22,2020-01-22,NaN,China,CHN,CN,National,0,NaN,NaN,Public Awareness Measures,Disseminating information related to COVID-19 ...,NaN,NaN,China,NaN,NaN,NaN,NaN,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),"National Government,Ministry/Department of Hea...",52.655965,51.59842,50.520158,194.0,http://wjw.wuhan.gov.cn/ztzl_28/fk/tzgg/202004...,2020-07-20,2020-07-20T21:13:39Z
4,R_1OoGDBSpbZQ02oANA,5468900,new_entry,original,NaN,NaN,"Xi'an, China announced that subway stations an...",2020-01-22,2020-01-22,NaN,China,CHN,CN,Municipal,0,Shaanxi,Xi'an,Hygiene,NaN,NaN,NaN,China,NaN,NaN,Shaanxi,Xi'an,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),Municipal Government,52.655965,51.59842,50.520158,194.0,http://www.sxdaily.com.cn/2020-01/30/content_8...,2020-07-16,2020-07-16T07:39:56Z
5,R_3j8nyNRePoxoLrQAj,7559732,new_entry,correction,NaN,NaN,"China, announced on January 22: to prevent and...",2020-01-22,2020-01-22,NaN,China,CHN,CN,National,0,NaN,NaN,Public Awareness Measures,Both Disseminating and Gathering information r...,NaN,NaN,China,NaN,NaN,NaN,NaN,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),"National Government,Ministry/Department of Hea...",52.655965,51.59842,50.520158,194.0,http://wjw.wuhan.gov.cn/zwgk_28/zcgz/202005/t2...,2020-07-20,2020-07-20T21:24:25Z
6,R_2YWyUcctdN6v06LNA,8059429,new_entry,original,NaN,NaN,"From January 22, patients with pneumonia cause...",2020-01-22,2020-01-22,NaN,China,CHN,CN,National,0,NaN,NaN,Other Policy Not Listed Above,NaN,China is to offer financial aid to patients wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),"National Government,Ministry/Department of Hea...",52.655965,51.59842,50.520158,194.0,http://www.xinhuanet.com/english/2020-01/28/c_...,2020-04-04,2020-04-04T03:52:30Z
7,R_30j9tBDjjkonFQwBs,9584758,new_entry,original,NaN,NaN,"Wuhan, China ordered the public to wear masks ...",2020-01-22,2020-01-22,NaN,China,CHN,CN,Municipal,1,Hubei,Wuhan,Health Resources,Masks,NaN,NaN,China,A geographical or administrative unit within a...,NaN,Hubei,Wuhan,NaN,All Residents (Citizen Residents +Foreign Resi...,Inbound/Outbound,Not Applicable,Mandatory with Legal Penalties (Jail Time),Municipal Government,52.655965,51.59842,50.520158,194.0,http://www.chinanews.com/gn/2020/01-22/9067622...,2020-04-03,2020-04-03T21:42:26Z


#### Let's compare the columns and figure out what we need

The meaning of codes can be found [here](https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md) (oxford) and [here](https://www.coronanet-project.org/Codebook_website.html) (coronanet)

In [22]:
df_us_ox.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode', 'Date',
       'C1_School closing', 'C1_Flag', 'C1_Notes', 'C2_Workplace closing',
       'C2_Flag', 'C2_Notes', 'C3_Cancel public events', 'C3_Flag', 'C3_Notes',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C4_Notes',
       'C5_Close public transport', 'C5_Flag', 'C5_Notes',
       'C6_Stay at home requirements', 'C6_Flag', 'C6_Notes',
       'C7_Restrictions on internal movement', 'C7_Flag', 'C7_Notes',
       'C8_International travel controls', 'C8_Notes', 'E1_Income support',
       'E1_Flag', 'E1_Notes', 'E2_Debt/contract relief', 'E2_Notes',
       'E3_Fiscal measures', 'E3_Notes', 'E4_International support',
       'E4_Notes', 'H1_Public information campaigns', 'H1_Flag', 'H1_Notes',
       'H2_Testing policy', 'H2_Notes', 'H3_Contact tracing', 'H3_Notes',
       'H4_Emergency investment in healthcare', 'H4_Notes',
       'H5_Investment in vaccines', 'H5_Notes', 'M1_Wildcard', 'M1_Notes',
       'ConfirmedC

In [143]:
df_corona_net_china.columns

Index(['record_id', 'policy_id', 'entry_type', 'correct_type', 'update_type',
       'update_level', 'description', 'date_announced', 'date_start',
       'date_end', 'country', 'ISO_A3', 'ISO_A2', 'init_country_level',
       'domestic_policy', 'province', 'city', 'type', 'type_sub_cat',
       'type_text', 'school_status', 'target_country', 'target_geog_level',
       'target_region', 'target_province', 'target_city', 'target_other',
       'target_who_what', 'target_direction', 'travel_mechanism', 'compliance',
       'enforcer', 'index_high_est', 'index_med_est', 'index_low_est',
       'index_country_rank', 'link', 'date_updated', 'recorded_date'],
      dtype='object')

( QUESTION: Should I use date_announced instead of date_start? )

For now, we only want province-level data

In [214]:
t = df_corona_net[df_corona_net.date_end.notna()]
t[t.country == 'China']

,record_id,policy_id,entry_type,correct_type,update_type,update_level,description,date_announced,date_start,date_end,country,ISO_A3,ISO_A2,init_country_level,domestic_policy,province,city,type,type_sub_cat,type_text,school_status,target_country,target_geog_level,target_region,target_province,target_city,target_other,target_who_what,target_direction,travel_mechanism,compliance,enforcer,index_high_est,index_med_est,index_low_est,index_country_rank,link,date_updated,recorded_date
81,R_27eQUD05p74fORiNA,3484811,update,original,End of Policy,NaN,Guizhou raised health emergency response to th...,2020-01-24,2020-01-24,2020-02-24,China,CHN,CN,Provincial,0,Guizhou,NaN,Declaration of Emergency,NaN,NaN,NaN,China,NaN,NaN,Guizhou,NaN,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),Provincial/State Government,53.063301,51.961999,50.848488,194.0,http://china.caixin.com/2020-01-29/101509411.html,2020-07-02,2020-07-02T02:01:25Z
82,R_oYvDmnyoZKwO4uJNA,3910940,update,original,End of Policy,NaN,Yunnan raised health emergency response to the...,2020-01-24,2020-01-24,2020-02-24,China,CHN,CN,Provincial,0,Yunnan,NaN,Declaration of Emergency,NaN,NaN,NaN,China,NaN,NaN,Yunnan,NaN,NaN,13,NaN,NaN,Mandatory (Unspecified/Implied),Provincial/State Government,53.063301,51.961999,50.848488,194.0,http://china.caixin.com/2020-01-29/101509411.html,2020-07-02,2020-07-02T01:59:04Z
83,R_2fuJdf9hxdIYx6dCn,4442029,new_entry,original,NaN,NaN,"Many tourist attractions (including museums, p...",2020-01-24,2020-01-24,2020-02-20,China,CHN,CN,Provincial,1,Jiangsu,NaN,Restriction and Regulation of Government Services,Parks,NaN,NaN,China,NaN,NaN,Jiangsu,NaN,NaN,All Residents (Citizen Residents +Foreign Resi...,NaN,NaN,Mandatory (Unspecified/Implied),"Provincial/State Government,Municipal Governme...",53.063301,51.961999,50.848488,194.0,http://news.yangtse.com/content/853705.html,2020-08-07,2020-08-07T23:32:22Z
84,R_2fuJdf9hxdIYx6dDx,4442029,new_entry,original,NaN,NaN,"Many tourist attractions (including museums, p...",2020-01-24,2020-01-24,2020-02-20,China,CHN,CN,Provincial,1,Jiangsu,NaN,Restriction and Regulation of Government Services,Tourist Sites,NaN,NaN,China,NaN,NaN,Jiangsu,NaN,NaN,All Residents (Citizen Residents +Foreign Resi...,NaN,NaN,Mandatory (Unspecified/Implied),"Provincial/State Government,Municipal Governme...",53.063301,51.961999,50.848488,194.0,http://news.yangtse.com/content/853705.html,2020-08-07,2020-08-07T23:32:22Z
85,R_2fuJdf9hxdIYx6dCy,4442029,new_entry,original,NaN,NaN,"Many tourist attractions (including museums, p...",2020-01-24,2020-01-24,2020-02-20,China,CHN,CN,Provincial,1,Jiangsu,NaN,Restriction and Regulation of Government Services,Public libraries,NaN,NaN,China,NaN,NaN,Jiangsu,NaN,NaN,All Residents (Citizen Residents +Foreign Resi...,NaN,NaN,Mandatory (Unspecified/Implied),"Provincial/State Government,Municipal Governme...",53.063301,51.961999,50.848488,194.0,http://news.yangtse.com/content/853705.html,2020-08-07,2020-08-07T23:32:22Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27837,R_1LoFyxwYEGP5Rx4Cj,9776954,new_entry,original,NaN,NaN,"From June 19, Hong Kong relaxed its regulation...",2020-06-16,2020-06-19,2020-07-02,China,CHN,CN,"No, it is at the national level",1,Hong Kong,NaN,Restriction and Regulation of Businesses,Other Non-Essential Businesses,NaN,NaN,China,NaN,NaN,NaN,NaN,NaN,All Residents (Citizen Residents +Foreign Resi...,NaN,NaN,"Mandatory with Legal Penalties (Jail Time),Man...","National Government,Police,Other (Please speci...",NaN,NaN,NaN,NaN,https://gia.info.gov.hk/general/202006/16/P202...,2020-06-22,2020-06-22T17:43:52Z
27838,R_PZh26LjKh2yiLtvNA,9826398,new_entry,original,NaN,NaN,"From June 19, Hong Kong relaxed the number of ...",2020-06-16,2020-06-19,2020-07-02,China,CHN,CN,"No, it is at the national level",1,Hong Kong,NaN,Restrictions of Mass Gatherings,NaN,NaN,NaN,China,NaN,NaN,NaN,NaN,NaN,All Residents (C

In [169]:
df = df_corona_net_china[['country', 'ISO_A3', 'province', 'date_start', '']]
df.head()

,country,ISO_A3,province,date_start
3,China,CHN,NaN,2020-01-22
4,China,CHN,Shaanxi,2020-01-22
5,China,CHN,NaN,2020-01-22
6,China,CHN,NaN,2020-01-22
7,China,CHN,Hubei,2020-01-22


We use a little web scraping to get the province code

In [37]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [40]:
url = "https://www.cottongen.org/data/nomenclatures/China_provinces"

# Openning connection and request page
uClient = uReq(url)
page_html = uClient.read()
uClient.close()

# Parse HTML
page_soup = soup(page_html, "html.parser")

# Grab each row
containers = page_soup.findAll("td")
len(containers)

72

In [49]:
containers[0]

<td class="rteright" height="20" style="height:20px;">Anhui Province  </td>

In [53]:
containers[1]

<td class="rtecenter">AH</td>

In [58]:
containers[2].text.strip() == 'Hong Kong Special Administrative Region' and containers[3].text.strip() == 'HK'

True

In [137]:
province_code = {}

In [138]:
words_to_remove = ["Province", "Municipality", "Autonomous", "Region", "Special", "Administrative", "Autonomous", "Zhuang", "Uyghur"]

def clean_up_region_name(name):
    for word in words_to_remove:
        name = name.replace(word, "")
    return name.strip()

In [139]:
for i in range(int(len(containers) / 2)):
    name = clean_up_region_name(containers[i * 2].text)
    code = containers[i * 2 + 1].text.strip()
    if name and code:
        province_code[name] = code

del province_code['Taiwan']
del province_code['Ningxia Hui']
province_code["Ningxia Hui Autonomous Region"] = "NX"